# 取数和运算函数相分离测试

In [12]:
import pandas as pd
import numpy as np
import akshare as ak
import os

C:\Users\jason\anaconda3\envs\riskfolio\lib\site-packages\akshare\__init__.py:2747: UserWarning: 为了支持更多特性，请将 Python 升级到 3.9.0 及以上版本！
  warnings.warn(


In [9]:
# 获取基金基本信息函数并储存到本地
def get_fund_basic_info(sw_index_list, filename):
    # 确保data目录存在
    data_dir = 'data'
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    
    # 构建完整的文件路径
    file_path = os.path.join(data_dir, filename)
    
    fund_df = pd.DataFrame()
    i = 0
    for code in sw_index_list:
        bars = ak.fund_individual_basic_info_xq(symbol=code)
        bars.index = bars['item']
        fund_df[code] = bars['value']
        i += 1
        print(f"\r已获取[{i}/{len(sw_index_list)}]支基金的数据", end="")
    fund_df.columns = sw_index_list
    
    # 将数据存储到CSV文件
    fund_df.to_csv(file_path)
    print(f"\n数据已存储到 {file_path}")

In [14]:
# 筛选符合年限的基金
def filter_funds_by_years(filename, year):
    # 构建完整的文件路径
    file_path = os.path.join('data', filename)
    
    # 从CSV文件中读取数据
    fund_df = pd.read_csv(file_path, index_col=0)
    
    # 获取某一年之前的基金列表
    fund_df_filtered = fund_df.loc['成立时间']
    # 将Series转换为DataFrame
    df = fund_df_filtered.reset_index()
    df.columns = ['index', 'date']
    # 将日期字符串转换为datetime对象
    df['date'] = pd.to_datetime(df['date'])
    df_before_data = df[df['date'].dt.year < year]

    # 输出最后的列表
    index_list = df_before_data['index'].tolist()
    return index_list

In [17]:
# 获取数据并存储到本地
def save_fund_price_to_local(index_list):
    for code in index_list:
        bars = ak.fund_open_fund_info_em(symbol=code, indicator="累计净值走势")
        bars['净值日期'] = pd.to_datetime(bars['净值日期']).dt.strftime('%Y-%m-%d')
        bars.index = pd.to_datetime(bars['净值日期'])
        bars.to_csv(os.path.join('data', f'{code}.csv'))
        print(f"已保存基金{code}的数据到本地")

In [19]:
# 从本地读取数据并合并
def read_and_merge_fund_price(index_list):
    price_df = pd.DataFrame()

    for code in index_list:
        file_path = os.path.join('data', f'{code}.csv')
        if os.path.exists(file_path):
            bars = pd.read_csv(file_path, index_col=0)
            price_df[code] = bars['累计净值']
        else:
            print(f"找不到基金{code}的本地数据")

    price_df.columns = index_list
    return price_df

In [10]:
sw_index_list = ['040046',
                     '007380',
                     '015016',
                     '013308',
                     ]

In [13]:
# 从接口获取数据并存储到CSV文件
store_fund_data_to_csv(sw_index_list, 'fund_data.csv')

已获取[4/4]支基金的数据
数据已存储到 data\fund_data.csv


In [15]:
year = 2024
index_list = read_fund_data_from_csv('fund_data.csv', year)

In [16]:
index_list

['040046', '007380', '015016', '013308']

In [18]:
save_fund_price_to_local(index_list)

已保存基金040046的数据到本地
已保存基金007380的数据到本地
已保存基金015016的数据到本地
已保存基金013308的数据到本地


In [20]:
price_df = read_and_merge_fund_price(index_list)

In [22]:
price_df

,040046,007380,015016,013308
净值日期,,,,
2013-08-02,1.000,NaN,NaN,NaN
2013-08-09,1.001,NaN,NaN,NaN
2013-08-16,1.001,NaN,NaN,NaN
2013-08-23,1.002,NaN,NaN,NaN
2013-08-26,1.002,NaN,NaN,NaN
...,...,...,...,...
2024-03-25,5.639,1.0349,1.514,0.8767
2024-03-26,5.615,1.0398,1.525,0.8845
2024-03-27,5.635,1.0335,1.531,0.8653


In [24]:
price_df.index

Index(['2013-08-02', '2013-08-09', '2013-08-16', '2013-08-23', '2013-08-26',
       '2013-08-27', '2013-08-28', '2013-08-29', '2013-08-30', '2013-09-02',
       ...
       '2024-03-18', '2024-03-19', '2024-03-20', '2024-03-21', '2024-03-22',
       '2024-03-25', '2024-03-26', '2024-03-27', '2024-03-28', '2024-03-29'],
      dtype='object', name='净值日期', length=2588)